<a href="https://www.kaggle.com/code/gabenazario/sql-murder-mystery?scriptVersionId=108121568" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# SQL Murder Mystery
## Can you find out whodunnit?

![SQL Murder Mystery](https://mystery.knightlab.com/174092-clue-illustration.png)

## There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
con = sqlite3.connect("../input/sql-murder-mystery-database/sql-murder-mystery.db")

In [4]:
cur = con.cursor()

In [5]:
tbl_names = pd.read_sql_query("SELECT * from sqlite_master WHERE type='table'", con)
tbl_names

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,"CREATE TABLE crime_scene_report (\n date integer,\n type text,\n description text,\n city text\n )"
1,table,drivers_license,drivers_license,3,"CREATE TABLE drivers_license (\n id integer PRIMARY KEY,\n age integer,\n height integer,\n eye_color text,\n hair_color text,\n gender text,\n plate_number text,\n car_make text,\n car_model text\n )"
2,table,person,person,4,"CREATE TABLE person (\n id integer PRIMARY KEY,\n name text,\n license_id integer,\n address_number integer,\n address_street_name text,\n ssn integer,\n FOREIGN KEY (license_id) REFERENCES drivers_license(id)\n )"
3,table,facebook_event_checkin,facebook_event_checkin,5,"CREATE TABLE facebook_event_checkin (\n person_id integer,\n event_id integer,\n event_name text,\n date integer,\n FOREIGN KEY (person_id) REFERENCES person(id)\n )"
4,table,interview,interview,6,"CREATE TABLE interview (\n person_id integer,\n transcript text,\n FOREIGN KEY (person_id) REFERENCES person(id)\n )"
5,table,get_fit_now_member,get_fit_now_member,7,"CREATE TABLE get_fit_now_member (\n id text PRIMARY KEY,\n person_id integer,\n name text,\n membership_start_date integer,\n membership_status text,\n FOREIGN KEY (person_id) REFERENCES person(id)\n )"
6,table,get_fit_now_check_in,get_fit_now_check_in,9,"CREATE TABLE get_fit_now_check_in (\n membership_id text,\n check_in_date integer,\n check_in_time integer,\n check_out_time integer,\n FOREIGN KEY (membership_id) REFERENCES get_fit_now_member(id)\n )"
7,table,income,income,10,"CREATE TABLE income (\n ssn integer PRIMARY KEY,\n annual_income integer\n )"
8,table,solution,solution,11,"CREATE TABLE solution (\n user integer,\n value text\n )"


In [6]:
report = pd.read_sql_query("SELECT * from crime_scene_report WHERE date = 20180115 AND type = 'murder' and city = 'SQL City'", con)
report

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [7]:
pd.read_sql_query("SELECT * FROM person WHERE address_street_name = 'Franklin Ave' AND  name LIKE 'Annabel%'", con)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [8]:
pd.read_sql_query("SELECT * FROM interview WHERE person_id = 16371", con)

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [9]:
pd.read_sql_query("SELECT * FROM get_fit_now_check_in WHERE check_in_date = 20180109", con)

,membership_id,check_in_date,check_in_time,check_out_time
0,X0643,20180109,957,1164
1,UK1F2,20180109,344,518
2,XTE42,20180109,486,1124
3,1AE2H,20180109,461,944
4,6LSTG,20180109,399,515
5,7MWHJ,20180109,273,885
6,GE5Q8,20180109,367,959
7,48Z7A,20180109,1600,1730
8,48Z55,20180109,1530,1700
9,90081,20180109,1600,1700


### There are too many to try to find out individually. Will pursue the other witness.

In [10]:
pd.read_sql_query("SELECT * FROM person WHERE address_street_name = 'Northwestern Dr' ORDER BY address_number DESC LIMIT 1", con)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [11]:
pd.read_sql_query("SELECT * FROM interview WHERE person_id = 14887", con)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [12]:
pd.read_sql_query("SELECT * FROM get_fit_now_member WHERE id LIKE '48Z55'", con) #Using list of members from first witness

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


In [13]:
pd.read_sql_query("SELECT * FROM drivers_license WHERE plate_number LIKE '%H42W%' AND gender = 'male'", con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [14]:
pd.read_sql_query("SELECT * FROM person WHERE license_id = 423327 OR license_id = 664760", con)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [15]:
pd.read_sql_query("SELECT * FROM interview WHERE person_id = 67318", con)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


### Jeremy Bowers, and he is a tattletale.

In [16]:
pd.read_sql_query("SELECT * FROM drivers_license WHERE car_model LIKE '%model%' AND gender = 'female' AND hair_color = 'red' and height BETWEEN 65 AND 67", con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [17]:
pd.read_sql_query("SELECT *, COUNT(person_id) AS 'Count' FROM facebook_event_checkin WHERE event_name = 'SQL Symphony Concert' AND date BETWEEN 20171201 AND 20171231 GROUP BY person_id HAVING Count(*) = 3", con)

,person_id,event_id,event_name,date,Count
0,24556,1143,SQL Symphony Concert,20171207,3
1,99716,1143,SQL Symphony Concert,20171206,3


In [18]:
pd.read_sql_query("SELECT * FROM person WHERE id = 24556 OR id = 99716", con)

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [19]:
suspect_info = pd.read_sql_query("SELECT * FROM person WHERE id = 99716", con)
suspect_info

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [20]:
#suspect_name = suspect_info.loc[:,'name'].to_list()
#cur.execute("INSERT INTO solution VALUES (1, ?);", suspect_name)

In [21]:
pd.read_sql_query(" SELECT value FROM Solution;", con)

,value


### File is read only, to ge the answers we have to follow this link - [SQL Murder Mystery](http://mystery.knightlab.com/#experienced) - and insert the name of our suspect.

Outcome - The murder is **Jeremy Bowers** and he was hired by **Miranda Priestly**.